In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spacextoy-dataanalysis-launchprediction/SpaceX Metadata.docx
/kaggle/input/spacextoy-dataanalysis-launchprediction/spacex_correlation.csv
/kaggle/input/spacextoy-dataanalysis-launchprediction/spacex_contracts_customers_known_revenue.csv
/kaggle/input/spacextoy-dataanalysis-launchprediction/spacex_encoded.csv
/kaggle/input/spacextoy-dataanalysis-launchprediction/spacex_financials_raw.xlsx
/kaggle/input/spacextoy-dataanalysis-launchprediction/spacex_extra_correlation.csv
/kaggle/input/spacextoy-dataanalysis-launchprediction/spacex.csv
/kaggle/input/spacextoy-dataanalysis-launchprediction/spacex_financials.csv


In [2]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime
# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [3]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

In [4]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [5]:
response = requests.get(spacex_url)

In [6]:
#print(response.content)
response.status_code

200

In [7]:
# Use json_normalize meethod to convert the json result into a dataframe
data =  pd.json_normalize(response.json())

In [8]:
data.head(1)

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN


In [9]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'flight_number', 'date_utc', 'fairings.reused']]
# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date
# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [11]:
#Global variable
BoosterVersion = []
BoosterVersion

[]

In [12]:
# Call getBoosterVersion
getBoosterVersion(data)

In [13]:
launch_dict = {'FlightNumber': list(data['flight_number']), 'Date': list(data['date']),
'ReusedFairings': list(data['fairings.reused']), 'BoosterVersion':BoosterVersion}

In [14]:
# Create a data from launch_dict
data_f9_fairings = pd.DataFrame.from_dict(launch_dict)

In [16]:
data_f9_fairings = data_f9_fairings[data_f9_fairings['BoosterVersion']!='Falcon 1']

In [17]:
data_f9_fairings['BoosterVersion'].value_counts()

Falcon 9        98
Falcon Heavy     3
Name: BoosterVersion, dtype: int64

In [15]:
data_f9_fairings = data_f9_fairings[data_f9_fairings['BoosterVersion']!='Falcon Heavy']

In [16]:
data_f9_fairings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 105
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   FlightNumber    103 non-null    int64 
 1   Date            103 non-null    object
 2   ReusedFairings  71 non-null     object
 3   BoosterVersion  103 non-null    object
dtypes: int64(1), object(3)
memory usage: 4.0+ KB


In [23]:
data_f9_fairings.head()

,FlightNumber,Date,ReusedFairings,BoosterVersion
0,1,2006-03-24,False,Falcon 1
1,2,2007-03-21,False,Falcon 1
2,3,2008-08-03,False,Falcon 1
3,4,2008-09-28,False,Falcon 1
4,5,2009-07-13,False,Falcon 1


In [18]:
data_f9_fairings.isnull().sum()

FlightNumber       0
Date               0
ReusedFairings    32
BoosterVersion     0
dtype: int64

In [19]:
#inspect null values
null_fairings = data_f9_fairings[data_f9_fairings.ReusedFairings.isnull()]
null_fairings

,FlightNumber,Date,ReusedFairings,BoosterVersion
5,6,2010-06-04,None,Falcon 9
6,7,2010-12-08,NaN,Falcon 9
7,8,2012-05-22,NaN,Falcon 9
8,9,2012-10-08,NaN,Falcon 9
9,10,2013-03-01,NaN,Falcon 9
13,14,2014-04-18,NaN,Falcon 9
17,18,2014-09-21,NaN,Falcon 9
18,19,2015-01-10,NaN,Falcon 9
21,22,2015-04-14,NaN,Falcon 9
23,24,2015-06-28,NaN,Falcon 9


In [20]:
# create dataframe of null values to save and handle missing data
# dates in API source of source (ElonX is source of r/SpaceX which is source of SpaceX API)
null_fairings = null_fairings['Date'].astype(str)

In [22]:
# save full dataframe from API request
# save null data subset from full dataframe
data_f9_fairings.to_csv('api_fairings.csv', index=False)
null_fairings.to_csv('null_fairings.csv', index=False)

...See: Fairing Null Handling